<a href="https://colab.research.google.com/github/atoms-ai/TECPLOT/blob/master/Create_circles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
from random import randint

# Settings
img_size = 1024
no_images = 1000
circle_radius_range = (20, 25)
hexagon_size = 100
teardrop_size = 20

# Helper function to create teardrop
def create_teardrop(center, size):
    # Teardrop is a 270 degree ellipse
    axes = (size, size*2)
    angle = 45
    start_angle = 0
    end_angle = 270
    color = (255,255,255)
    return cv2.ellipse(np.zeros((img_size,img_size,3), np.uint8), center, axes, angle, start_angle, end_angle, color, -1)

# Function to check if a new circle overlaps with existing circles
def check_overlap(new_circle, existing_circles):
    for circle in existing_circles:
        dx = circle[0] - new_circle[0]
        dy = circle[1] - new_circle[1]
        distance = np.sqrt(dx * dx + dy * dy)
        if distance < circle[2] + new_circle[2]:
            return True
    return False

# Create directory for images
if not os.path.exists('images'):
    os.makedirs('images')

# Create directory for labels
if not os.path.exists('labels'):
    os.makedirs('labels')

# Create directory for csv files
if not os.path.exists('csv_files'):
    os.makedirs('csv_files')

# Dataframe to store all shapes info
df_shapes = pd.DataFrame(columns=['image_id', 'shape', 'center_x', 'center_y', 'size', 'orientation'])

for i in range(no_images):
    # Create image with light gray background
    image = np.full((img_size, img_size, 3), fill_value=175, dtype=np.uint8)

    # Create segmented label images
    segment_circles = np.zeros((img_size, img_size, 3), dtype=np.uint8)
    segment_circle_centers = np.zeros((img_size, img_size, 3), dtype=np.uint8)

    # Shape data for current image
    image_shapes = []

    # Generate circles
    circles = []
    for _ in range(randint(1, 10)):
        while True:
            radius = randint(*circle_radius_range)
            center = (randint(radius, img_size-radius), randint(radius, img_size-radius))
            if i < 500 or not check_overlap((center[0], center[1], radius), circles):  # Avoid overlap in first 500 images
                break

        circles.append((center[0], center[1], radius))
        cv2.circle(image, center, radius, (0,0,0), -1)
        cv2.circle(segment_circles, center, radius, (255,255,255), -1)
        cv2.circle(segment_circle_centers, center, 5, (255,255,255), -1)
        image_shapes.append(('circle', center, radius, None))

    # Generate hexagons
    for _ in range(randint(1, 10)):
        center = (randint(hexagon_size, img_size-hexagon_size), randint(hexagon_size, img_size-hexagon_size))
        hexagon = np.array([[center[0], center[1]-hexagon_size],
                            [center[0]+hexagon_size, center[1]-hexagon_size//2],
                            [center[0]+hexagon_size, center[1]+hexagon_size//2],
                            [center[0], center[1]+hexagon_size],
                            [center[0]-hexagon_size, center[1]+hexagon_size//2],
                            [center[0]-hexagon_size, center[1]-hexagon_size//2]], np.int32)
        cv2.fillPoly(image, [hexagon], (0,0,0))
        image_shapes.append(('hexagon', center, hexagon_size, None))

    # Generate teardrops
    for _ in range(randint(1, 10)):
        center = (randint(teardrop_size, img_size-teardrop_size), randint(teardrop_size, img_size-teardrop_size))
        image += create_teardrop(center, teardrop_size)
        image_shapes.append(('teardrop', center, teardrop_size, 45))

    # Add noise
    noise = np.random.normal(0, 1, image.shape).astype(np.uint8)
    image = cv2.add(image, noise)

    # Save image
    image_name = f'images_{str(i+1).zfill(4)}'
    cv2.imwrite(f'images/{image_name}.png', image)

    # Save labels
    cv2.imwrite(f'labels/segment_circles_{str(i+1).zfill(4)}.png', segment_circles)
    cv2.imwrite(f'labels/segment_circle_centers_{str(i+1).zfill(4)}.png', segment_circle_centers)

    # Save shapes data to dataframe
    df_shapes = df_shapes.append(pd.DataFrame([(image_name, shape[0], shape[1][0], shape[1][1], shape[2], shape[3] if shape[3] is not None else 0) for shape in image_shapes], columns=df_shapes.columns))

# Save all shapes data to csv file
df_shapes.to_csv('csv_files/shapes.csv', index=False)


<ipython-input-2-85a975554dda>:106: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_shapes = df_shapes.append(pd.DataFrame([(image_name, shape[0], shape[1][0], shape[1][1], shape[2], shape[3] if shape[3] is not None else 0) for shape in image_shapes], columns=df_shapes.columns))
<ipython-input-2-85a975554dda>:106: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_shapes = df_shapes.append(pd.DataFrame([(image_name, shape[0], shape[1][0], shape[1][1], shape[2], shape[3] if shape[3] is not None else 0) for shape in image_shapes], columns=df_shapes.columns))
<ipython-input-2-85a975554dda>:106: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_shapes = df_shapes.append(pd.DataFrame([(image_name, shape[0], shape[1][0], shape[1][1